In [43]:
# Import dependencies
import pandas as pd
import numpy as np

from bokeh.plotting import figure, show
from bokeh.io.output import output_notebook
from bokeh.models import DatetimeTickFormatter

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

import pmdarima as pm
from prophet import Prophet

In [3]:
store_sales_train = pd.read_csv(
    'train.csv',
    usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)

store_sales_train['date'] = store_sales_train.date.dt.to_period('D')

store_sales_test = pd.read_csv(
    'test.csv',
    usecols=['store_nbr', 'family', 'date', 'onpromotion'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)

store_sales_test['date'] = store_sales_test.date.dt.to_period('D')

store_sales_train

,date,store_nbr,family,sales,onpromotion
0,2013-01-01,1,AUTOMOTIVE,0.000000,0
1,2013-01-01,1,BABY CARE,0.000000,0
2,2013-01-01,1,BEAUTY,0.000000,0
3,2013-01-01,1,BEVERAGES,0.000000,0
4,2013-01-01,1,BOOKS,0.000000,0
...,...,...,...,...,...
3000883,2017-08-15,9,POULTRY,438.132996,0
3000884,2017-08-15,9,PREPARED FOODS,154.552994,1
3000885,2017-08-15,9,PRODUCE,2419.729004,148
3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000000,8


In [4]:
family_sales_train = (
    store_sales_train
    .groupby(['family', 'date'])
    .mean()
    .unstack('family')
    .loc["2017"]
)

family_sales_train.head()

sales                                              \
family     AUTOMOTIVE BABY CARE     BEAUTY    BEVERAGES     BOOKS   
date                                                                
2017-01-01   0.092593  0.037037   0.055556    74.222221  0.000000   
2017-01-02  11.481482  0.259259  11.648149  6208.055664  0.481481   
2017-01-03   8.296296  0.296296   7.185185  4507.814941  0.814815   
2017-01-04   6.833333  0.333333   6.888889  3911.833252  0.759259   
2017-01-05   6.333333  0.351852   5.925926  3258.796387  0.407407   

                                                                           \
family     BREAD/BAKERY CELEBRATION     CLEANING        DAIRY        DELI   
date                                                                        
2017-01-01     9.084685    0.129630     7.500000    11.518518    3.629167   
2017-01-02   844.836304   14.203704  2233.648193  1545.000000  539.114807   
2017-01-03   665.124084   10.629630  1711.907349  1204.203735  404.300079   
2017-01-04   594.160583   11.185185  1508.036987  1107.796265  309.397675   
2017-01-05   495.511597   12.444445  1241.833374   829.277771  260.776489   

            ... onpromotion                                       \
family      ...   MAGAZINES     MEATS PERSONAL CARE PET SUPPLIES   
date        ...                                                    
2017-01-01  ...         0.0  0.018519      0.111111     0.018519   
2017-01-02  ...         0.0  0.462963     10.592593     0.537037   
2017-01-03  ...         0.0  0.481481      9.722222     0.444444   
2017-01-04  ...         0.0  0.370370     12.037037     0.444444   
2017-01-05  ...         0.0  8.981481      5.666667     0.000000   

                                                                         \
family     PLAYERS AND ELECTRONICS   POULTRY PREPARED FOODS     PRODUCE   
date                                                                      
2017-01-01                     0.0  0.000000       0.037037    0.129630   
2017-01-02                     0.0  0.259259       1.166667    5.629630   
2017-01-03                     0.0  0.388889       1.351852   56.296296   
2017-01-04                     0.0  0.296296       5.444444  101.277778   
2017-01-05                     0.0  0.296296       0.907407    5.018519   

                                                 
family     SCHOOL AND OFFICE SUPPLIES   SEAFOOD  
date                                             
2017-01-01                        0.0  0.000000  
2017-01-02                        0.0  0.407407  
2017-01-03                        0.0  0.407407  
2017-01-04                        0.0  0.333333  
2017-01-05                        0.0  0.444444  

[5 rows x 66 columns]

In [5]:
beauty_sales = family_sales_train[(      'sales',                     'BEAUTY')]

In [6]:
X1_train = np.arange(len(beauty_sales)).reshape(-1, 1)
X1_train, X1_val = train_test_split(X1_train, test_size=.3, shuffle=False)

X2_train = beauty_sales.to_numpy().reshape(-1, 1)
X2_train, X2_val = train_test_split(X2_train, test_size=.3, shuffle=False)

y_train = beauty_sales.values.reshape(-1, 1)
y_train, y_val = train_test_split(y_train, test_size=.3, shuffle=False)

date_index_train, date_index_val = train_test_split(beauty_sales.index, test_size=.3, shuffle=False)

In [7]:
class HybridModel:
    def __init__(self, model_1, model_2):
        self.model_1 = model_1
        self.model_2 = model_2
        
    def fit(self, X_1, X_2, y):
        self.model_1 = self.model_1.fit(X_1, y)
        y_1_pred = self.model_1.predict(X_1)
        self.model_2 = self.model_2.fit(X_2, y - y_1_pred)
        
    def predict(self, X_1, X_2):
        y_1_pred = self.model_1.predict(X_1)
        y_2_pred = self.model_2.predict(X_2)
        
        return y_1_pred.flatten() + y_2_pred.flatten()

In [8]:
lr = LinearRegression()
xgbr = XGBRegressor()
hm = HybridModel(lr, xgbr)
hm.fit(X1_train, X2_train, y_train)

pred_train = hm.predict(X1_train, X2_train)
pred_val = hm.predict(X1_val, X2_val)

In [9]:
output_notebook()

# create a new plot with a title and axis labels
p = figure(title="Simple line example", x_axis_label='Date', y_axis_label='Sales', width=900, height=300)

# add a line renderer with legend and line thickness to the plot
p.line(
    beauty_sales.index,
    beauty_sales,  
    line_width=1)

p.line(
    date_index_train,
    pred_train.flatten(),  
    line_width=1, 
    line_color="red")


p.line(
    date_index_val,
    pred_val.flatten(),  
    line_width=1, 
    line_color="orange")

p.xaxis[0].formatter = DatetimeTickFormatter(days='%m/%d')

# show the results
show(p, output_notebook=True)

mae = sum(abs(pred_val - y_val.flatten())) / len(pred_val)
print(f"Mean Absolute Error: {round(mae, 3)}")

Loading BokehJS ...

Mean Absolute Error: 0.317


In [24]:
# Fit auto ARIMA
model = pm.auto_arima(y_train.flatten(), seasonal=True, m=12)

# make your forecasts
forecasts = model.predict(X1_val.shape[0])  # predict N steps into the future

y2_pred_val = hm.model_2.predict(X2_val)

pred_val = forecasts + y2_pred_val.flatten()

In [26]:
output_notebook()

# create a new plot with a title and axis labels
p = figure(title="Simple line example", x_axis_label='Date', y_axis_label='Sales', width=900, height=300)

# add a line renderer with legend and line thickness to the plot
p.line(
    beauty_sales.index,
    beauty_sales,  
    line_width=1)

p.line(
    date_index_train,
    pred_train.flatten(),  
    line_width=1, 
    line_color="red")


p.line(
    date_index_val,
    pred_val,  
    line_width=1, 
    line_color="orange")

p.xaxis[0].formatter = DatetimeTickFormatter(days='%m/%d')

# show the results
show(p, output_notebook=True)

mae = sum(abs(pred_val - y_val.flatten())) / len(pred_val)
print(f"Mean Absolute Error: {round(mae, 3)}")

Loading BokehJS ...

Mean Absolute Error: 0.607


In [37]:
df_prophet = beauty_sales.to_frame().reset_index()
df_prophet = df_prophet[:N]
df_prophet.columns = ["ds", "y"]
df_prophet['ds'] = df_prophet.ds.apply(lambda x: x.to_timestamp())

m = Prophet()
m.fit(df_prophet)

future = m.make_future_dataframe(periods=20)

forecast = m.predict(future)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [42]:
output_notebook()

# create a new plot with a title and axis labels
p = figure(title="Simple line example", x_axis_label='Date', y_axis_label='Sales', width=900, height=300)

# add a line renderer with legend and line thickness to the plot
p.line(
    beauty_sales.index,
    beauty_sales,  
    line_width=1)

# p.line(
#     date_index_train,
#     pred_train.flatten(),  
#     line_width=1, 
#     line_color="red")


p.line(
    date_index_val,
    forecast['yhat'][:69],  
    line_width=1, 
    line_color="orange")

p.xaxis[0].formatter = DatetimeTickFormatter(days='%m/%d')

# show the results
show(p, output_notebook=True)

mae = sum(abs(forecast['yhat'][:69] - y_val.flatten())) / len(pred_val)
print(f"Mean Absolute Error: {round(mae, 3)}")

Loading BokehJS ...

Mean Absolute Error: 3.326


In [29]:
# Prophet would benefits from having more training data
family_sales_train_prophet = (
    store_sales_train
    .groupby(['family', 'date'])
    .mean()
    .unstack('family')
)

beauty_sales_prophet = family_sales_train_prophet[(      'sales',                     'BEAUTY')]

N = int(len(beauty_sales_prophet) * .7)

In [39]:
df_prophet = beauty_sales_prophet.to_frame().reset_index()
df_prophet = df_prophet[:N]
df_prophet.columns = ["ds", "y"]
df_prophet['ds'] = df_prophet.ds.apply(lambda x: x.to_timestamp())

m = Prophet()
m.fit(df_prophet)

future = m.make_future_dataframe(periods=20)

forecast = m.predict(future)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [41]:
output_notebook()

# create a new plot with a title and axis labels
p = figure(title="Prophet", x_axis_label='Date', y_axis_label='Sales', width=900, height=300)

# add a line renderer with legend and line thickness to the plot
p.line(
    beauty_sales.index,
    beauty_sales,  
    line_width=1)

p.line(
    date_index_val,
    forecast['yhat'][:69],  
    line_width=1, 
    line_color="orange")

p.xaxis[0].formatter = DatetimeTickFormatter(days='%m/%d')

# show the results
show(p, output_notebook=True)

mae = sum(abs(forecast['yhat'][:69] - y_val.flatten())) / len(pred_val)
print(f"Mean Absolute Error: {round(mae, 3)}")

Loading BokehJS ...

Mean Absolute Error: 3.326


More data does not help Prophet in this case.

# Summary MAE:

    Hybrid Model (Linear Regression + XGBClassifier): 0.317
    
    Hybrid Model (ARIMA + XGBClassifier): 0.607
    
    Hybrid Model (Prophet + XGBClassifier): 3.326
    
    Hybrid Model (Prophet with available data + XGBClassifier): 3.326